In [ ]:
from ipydatagrid import DataGrid, Filter, Sort, TextRenderer, BarRenderer, VegaExpr

In [ ]:
from json import load

with open('./cars.json') as fobj:
    data = load(fobj)

In [ ]:
from bqplot import LinearScale, ColorScale, OrdinalColorScale, OrdinalScale

renderers = {
    'Acceleration': BarRenderer(
        horizontal_alignment='center',
        bar_color=ColorScale(min=0, max=20, scheme='viridis'),
        value=LinearScale(min=0, max=20)
    ),
    'Displacement': TextRenderer(
        text_color=ColorScale(min=97, max=455), 
        font=VegaExpr("value > 400 ? '16px sans-serif' : '12px sans-serif'")
    ),
    'Horsepower': TextRenderer(
        background_color=VegaExpr("value < 150 ? 'red' : 'green'")
    ),
    'Name': TextRenderer(
        background_color=VegaExpr("indexof(value, 'chevrolet') !== -1 ? 'green' : 'white'")
    ),
    'Origin': TextRenderer(
        background_color=OrdinalColorScale(domain=['USA', 'Japan', 'Europe']),
        horizontal_alignment=VegaExpr("value == 'USA' ? 'right' : 'left'")
    ),
    'Year': TextRenderer(background_color='green'),
}

datagrid = DataGrid(data=data, base_row_size=32, base_column_size=100, renderers=renderers)
datagrid

In [ ]:
renderers['Horsepower'].background_color.value = "value < 150 ? 'orange' : 'blue'"

In [ ]:
datagrid.transform([Filter('Origin', '=', 'Europe'), Sort('Displacement')])

In [ ]:
datagrid.revert()

In [ ]:
datagrid.transform([Filter('Origin', '=', 'USA'), Filter('Acceleration', '<', 13), Sort('Acceleration')])

# Perfomance test: A million cells with ipydatagrid

In [ ]:
from random import uniform

def create_random_data(n_rows=100, n_columns=100):
    data = {'data': [], 'schema': {}}

    data['data'] = [[uniform(0, 1) for c in range(n_columns)] for r in range(n_rows)]
    data['schema']['fields'] = [{'name': str(c), type:'number'} for c in range(n_columns)]
    
    return data

In [ ]:
huge_data = create_random_data(1000, 1000)

conditional_renderer = VegaExpr("value < 0.5 ? '#fc8403' : 'white'")

conditional_huge_datagrid = DataGrid(data=huge_data, default_renderer=TextRenderer(background_color=conditional_renderer))
conditional_huge_datagrid

In [ ]:
from ipywidgets import FloatSlider, Dropdown, ColorPicker, HBox

operator_dropdown = Dropdown(
    options=['<', '>'],
    value='<'
)
reference_slider = FloatSlider(value=0.5, min=0, max=1)
output_colorpicker = ColorPicker(
    value='#fc8403'
)

def on_change(*args, **kwargs):
    conditional_renderer.value = "value {operator} {reference} ? '{color}' : 'white'".format(
        operator=operator_dropdown.value,
        reference=reference_slider.value,
        color=output_colorpicker.value
    )

operator_dropdown.observe(on_change, 'value')
reference_slider.observe(on_change, 'value')
output_colorpicker.observe(on_change, 'value')

HBox((operator_dropdown, reference_slider, output_colorpicker))

## Bar renderer

In [ ]:
from bqplot import LinearScale, ColorScale
from ipydatagrid import DataGrid, BarRenderer

linear_scale = LinearScale(min=0, max=1)
color_scale = ColorScale(min=0, max=1)
bar_renderer = BarRenderer(bar_color=color_scale, value=linear_scale, bar_horizontal_alignment='center')

huge_datagrid = DataGrid(data=create_random_data(), default_renderer=bar_renderer)

In [ ]:
huge_datagrid

In [ ]:
from ipywidgets import FloatSlider, link

slider = FloatSlider(description='Scale: ', value=linear_scale.max, min=0, max=1)
link((color_scale, 'min'), (slider, 'value'))
link((linear_scale, 'min'), (slider, 'value'))

slider

In [ ]:
color_scale.scheme = 'magma'